Stajesz przed nowym zadaniem, nowy product manager poprosił Cię o pomoc w obliczeniu i wizualizacji nowego KPI, który ma
pomóc ulepszyć proces dostarczania rezultatów do klienta. Jego definicja to: Ilość zaakceptowanych błędów na godzinę w 
pierwszych 6-ciu godzinach od rozpoczęcia cyklu testowego. PM przekazał Ci, że dla uproszczenia jako początek cyklu testowego możesz uznać czas pierwszego zgłoszonego błędu. 

Jeżeli uda Ci się pokazać tą metrykę, PM napewno doceni analizę na tych danych - ponieważ on nie ma na to czasu.

Stwórz widok pokazujący powyższą metryke i zaproponuj dodatkowe wykresy z które mogą być przydatne dla Product Manager'a.

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

In [84]:
data = pd.read_csv('cycle_data.csv')

In [45]:
data.sample(10)

,issue_code_,created_at,severity,resolution,affected_components,creator_id
64,CYCLE-10-65,13/04/2021 17:01:19,Medium,Confirmed (S),{Profile},5271.0
1,CYCLE-10-2,13/04/2021 15:40:15,Low,Confirmed (S),{Signup},8124.0
12,CYCLE-10-13,NaN,Medium,Confirmed (S),{Sign-up},5271.0
56,CYCLE-10-57,13/04/2021 16:40:33,Medium,Confirmed (!),{Content},4487.0
39,CYCLE-10-40,13/04/2021 16:19:52,Medium,Confirmed (S),{Content},8124.0
70,CYCLE-10-71,13/04/2021 17:08:05,Medium,Confirmed (S),{Workouts},4487.0
11,CYCLE-10-12,13/04/2021 15:49:31,High,Confirmed (S),{Signup},1066.0
10,CYCLE-10-11,13/04/2021 15:49:22,Medium,Confirmed (S),{Profile},3230.0
83,CYCLE-10-84,13/04/2021 19:01:57,High,Duplicate,{Login},6435.0
36,CYCLE-10-37,13/04/2021 16:14:44,High,Confirmed (S),{Kit},3230.0


In [46]:
data.severity.unique()

array(['Low', 'High', 'Medium', 'Critical'], dtype=object)

In [47]:
data.resolution.unique()

array(['Confirmed (S)', 'Duplicate', 'Expected Behaviour',
       'Confirmed (!)', 'Invalid - not tester error', 'Not in Scope',
       'No tester response', 'Confirm (S)'], dtype=object)

In [48]:
data.affected_components.unique()

array(['{Signup}', '{Profile}', 'Profile', '{Sign-up}', '{Content}',
       '{Challenge}', '{Workouts}', '{Settings}', '{Kit}', '{Guides}',
       '{Meals}', '{Login}'], dtype=object)

In [85]:
# First step will be to clear the data.
# I replace rows where resolution status is 'Confirmed (!)' and 'Confirm (S)' with status 'Confirmed (S)', because it's 
# the same category 

data["resolution"].replace({"Confirmed (!)": "Confirmed (S)", "Confirm (S)": "Confirmed (S)"}, inplace=True)
data.resolution.unique()

array(['Confirmed (S)', 'Duplicate', 'Expected Behaviour',
       'Invalid - not tester error', 'Not in Scope', 'No tester response'],
      dtype=object)

In [50]:
data.isnull().sum()

issue_code_            0
created_at             1
severity               0
resolution             0
affected_components    0
creator_id             1
dtype: int64

In [86]:
# Drop rows where NaN is in 'created_at' column, because we need this data to check if error was created within first 
# 6 hours of testing cycle.

data.dropna(subset=['created_at'], inplace=True)
data['created_at'].isnull().sum()

0

In [52]:
data.head(10)

,issue_code_,created_at,severity,resolution,affected_components,creator_id
0,CYCLE-10-1,13/04/2021 15:35:36,Low,Confirmed (S),{Signup},8124.0
1,CYCLE-10-2,13/04/2021 15:40:15,Low,Confirmed (S),{Signup},8124.0
2,CYCLE-10-3,13/04/2021 15:41:28,Low,Confirmed (S),{Signup},1066.0
3,CYCLE-10-4,13/04/2021 15:41:38,High,Confirmed (S),{Signup},4075.0
4,CYCLE-10-5,13/04/2021 15:43:07,Medium,Confirmed (S),{Profile},3230.0
5,CYCLE-10-6,13/04/2021 15:43:32,Low,Confirmed (S),{Signup},5271.0
6,CYCLE-10-7,13/04/2021 15:46:35,Low,Confirmed (S),{Signup},4693.0
7,CYCLE-10-8,13/04/2021 15:46:44,Medium,Confirmed (S),Profile,8058.0
8,CYCLE-10-9,13/04/2021 15:46:53,High,Confirmed (S),{Profile},3230.0
9,CYCLE-10-10,13/04/2021 15:47:03,Low,Duplicate,{Signup},1066.0


In [37]:
type(data['created_at'][0])

str

In [87]:
# I want to change data type in column "created_at" to datetime 
# There is an error appearing because of wrong format in one row, so I'm going to change this

data[data['created_at'] == '04/13/2021 16:25:40']

,issue_code_,created_at,severity,resolution,affected_components,creator_id
44,CYCLE-10-45,04/13/2021 16:25:40,High,Confirmed (S),{Workouts},4693.0


In [93]:
data.at[44, 'created_at'] = '13/04/2021 16:25:40'
data.loc[44, 'created_at']

'13/04/2021 16:25:40'

In [94]:
# Change format 
data['created_at'] = pd.to_datetime(data['created_at'], format='%d/%m/%Y %H:%M:%S')

In [95]:
type(data['created_at'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [96]:
data.head()

,issue_code_,created_at,severity,resolution,affected_components,creator_id
0,CYCLE-10-1,2021-04-13 15:35:36,Low,Confirmed (S),{Signup},8124.0
1,CYCLE-10-2,2021-04-13 15:40:15,Low,Confirmed (S),{Signup},8124.0
2,CYCLE-10-3,2021-04-13 15:41:28,Low,Confirmed (S),{Signup},1066.0
3,CYCLE-10-4,2021-04-13 15:41:38,High,Confirmed (S),{Signup},4075.0
4,CYCLE-10-5,2021-04-13 15:43:07,Medium,Confirmed (S),{Profile},3230.0


In [97]:
# Now I'm going to create a new column 'cycle_hour' with information about hour of testing cycle (from 1 to 6) in which error appeared
start_date = data.loc[0, 'created_at']
end_date = start_date + pd.DateOffset(hours=1)
start_date, end_date

(Timestamp('2021-04-13 15:35:36'), Timestamp('2021-04-13 16:35:36'))

In [102]:
for i in range(6):
    data.loc[(data['created_at'] >= start_date + pd.DateOffset(hours=i)) & (data['created_at'] < end_date + pd.DateOffset(hours=i)), 'cycle_hour'] = i+1

In [103]:
# Fill missing values with 7 - for every issue that appeared after 6 hours.
data['cycle_hour'].fillna(7.0, inplace=True)

In [104]:
data['cycle_hour'].unique()

array([1., 2., 3., 4., 5., 6., 7.])